# University of Iowa Formatting


To do list here.

In [1]:
# Libraries

from postal.expand import expand_address
import pandas as pd
import pickle

In [2]:
# Directories

dataDir = '../../data/external/ui-police/'

arrestsRaw = dataDir + 'docket.csv'
processDir = '../../data/processed/ui-police/'

In [3]:
# addresses

def formatAddr(raw):
    '''Function to format an address only if the raw string is valid.
    Input: String containing address
    Output: Normalized address'''
    
    # Error checking
    if ((type(raw) != str) | (len(raw) == 0)):
        return False
    
    # Prepare and get info
    query = raw + ', Iowa City, Iowa'
    expanded = expand_address(query)
    
    # return normalized address or notify
    directions = ['nb', 'eb', 'sb', 'wb']
    if (len(expanded) > 0):
        bestCand = expanded[0]
        if (bestCand[:2]) in directions:
            return bestCand[3:]
        else:
            return bestCand
    else:
        print (raw + ' did not expand!')

In [4]:
# read in: add cols, parse dates, ...

arrDF = pd.read_csv(arrestsRaw)
arrDF.rename(columns={'arrLocation': 'incAddr', 'date': 'incDate', 'time': 'incTime', 'address': 'homeAddr'}, inplace=True)

arrDF.head(2)

,incDate,name,age,homeAddr,incAddr,date2,chargeCode,charge,incTime,homeCity,homeState,homeZip
0,10/12/16,"KESSELL, BRANDON GAGE",18,1232 BURGE HALL,HANCHER FOOT BRIDGE EAST,NaN,124.401,POSSESSION:CONTROL SUBSTANCE,1:13,IOWA CITY,IA,52242
1,10/12/16,"KESSELL, BRANDON GAGE",18,1232 BURGE HALL,HANCHER FOOT BRIDGE EAST,NaN,124.414,DRUG PARAPHERNALIA,1:13,IOWA CITY,IA,52242


In [6]:
# read the dates and construct a time stamp
arrDF.incDate = pd.to_datetime(arrDF.incDate) # date

In [7]:
# address normalization
arrDF.incAddr = arrDF.incAddr.str.lower()
arrDF.incAddr = arrDF.incAddr.apply(lambda x: formatAddr(x))

In [14]:
print arrDF.name.value_counts().head(20)

JUVENILE                   497
INIAMA, UWEM KOKO           81
VOGELSANG, VINCENT JAY      76
ROBERTS, RICKIE ALLEN       45
MURDOCK, ROBERT  ALLEN      26
HINES, CLIFFORD CLYDE       26
ALDRICH, PATTY  KAY         21
PATTON, DANYELLE NICOLE     19
WARSON, MANDY RAE           19
RODDEN, RICK  EDWARD        17
RING, KENNETH DALE          15
YOUNG, BRETT  DAVID         15
ODEAN, JACOB JAMES          14
HENDRIK, NICODEMUS          13
HUA, JAMES THAI             13
OLLINGER, GUS  DANIEL       13
LEIBOLD, ALEC JORDAN        13
ADELMUND, GARY  ALAN        12
DAVIDSON, GEORGE ALLAN      12
YANG, SHUJUN                11
Name: name, dtype: int64


In [ ]:
# save that ish
arrDF.to_pickle((pro))